In [1]:
import torch.nn as nn
from torch.functional import F
class CNN(nn.Module):
    def __init__(self, vocab_size, embedding_dim, n_filters, filter_sizes, output_dim, 
                 dropout, pad_idx):
        
        super().__init__()
        
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        
        self.convs = nn.ModuleList([
                                    nn.Conv2d(in_channels = 1, 
                                              out_channels = n_filters, 
                                              kernel_size = (fs, embedding_dim)) 
                                    for fs in filter_sizes
                                    ])
        
        self.fc = nn.Linear(len(filter_sizes) * n_filters, output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, text):
        
        #text = [sent len, batch size]
        
        text = text.permute(1, 0)
                
        #text = [batch size, sent len]
        
        embedded = self.embedding(text)
                
        #embedded = [batch size, sent len, emb dim]
        
        embedded = embedded.unsqueeze(1)
        
        #embedded = [batch size, 1, sent len, emb dim]
        
        conved = [F.relu(conv(embedded)).squeeze(3) for conv in self.convs]
            
        #conv_n = [batch size, n_filters, sent len - filter_sizes[n]]
        
        pooled = [F.max_pool1d(conv, conv.shape[2]).squeeze(2) for conv in conved]
        
        #pooled_n = [batch size, n_filters]
        
        cat = self.dropout(torch.cat(pooled, dim = 1))

        #cat = [batch size, n_filters * len(filter_sizes)]
            
        return self.fc(cat)


In [2]:
INPUT_DIM =  20002
EMBEDDING_DIM = 100
N_FILTERS = 100
FILTER_SIZES = [2,3,4]
OUTPUT_DIM = 6
DROPOUT = 0.5
PAD_IDX =  1

model = CNN(INPUT_DIM, EMBEDDING_DIM, N_FILTERS, FILTER_SIZES ,OUTPUT_DIM, DROPOUT, PAD_IDX)

In [3]:
import torch
model.load_state_dict(torch.load('tut2-model.pt'))

#test_loss, test_acc = evaluate(model, test_iterator, criterion)

<All keys matched successfully>

In [87]:
import torchtext.vocab as vocab
from torchtext import data

custom_embeddings = vocab.Vectors(name = 'custom_embeddings/embeddings_conv.txt',
                                  cache = 'custom_embeddings')

 

In [135]:
emdeding_dict = custom_embeddings.stoi

In [147]:
custom_embeddings.vectors[custom_embeddings.stoi['<unk>']]

tensor([-5.0865e-02,  1.1826e-01,  2.5153e-01, -9.5067e-02, -9.5174e-02,
         1.2393e-01, -1.4499e-01,  5.6593e-02,  9.0250e-02,  3.1946e-03,
        -8.6432e-03, -8.3976e-02,  3.7212e-02, -7.1502e-02,  1.2413e-01,
        -8.3172e-03,  2.7153e-02, -3.3595e-02, -4.7731e-03,  2.6655e-01,
         1.8346e-01, -2.8661e-02, -9.5511e-02, -1.0686e-02,  3.5756e-01,
        -1.6515e-02, -4.6183e-02, -8.7500e-02,  1.3388e-01,  3.9147e-04,
        -3.1318e-02,  9.4908e-02,  2.0891e-02,  6.6893e-02, -1.6742e-02,
         8.6748e-02,  2.0689e-02,  4.6168e-02, -4.0032e-02, -6.3711e-02,
        -1.7659e-01, -6.8515e-02,  4.6157e-02, -1.2917e-01,  1.4088e-02,
        -3.6902e-02,  2.8122e-02, -7.9011e-02,  4.2249e-02, -7.9993e-02,
         3.7146e-02,  1.0368e-02,  5.1287e-02,  9.8828e-02, -4.1137e-01,
        -1.3085e-01,  1.6571e-02, -7.9508e-02,  1.8312e-01, -1.7528e-03,
        -4.8732e-02,  6.4475e-02, -2.7902e-01, -1.7228e-01,  2.7803e-01,
        -1.2206e-02,  2.9109e-01,  6.2339e-02,  1.6

In [95]:
nlp.tokenizer("kir quiet or kill you bitch")

kir quiet or kill you bitch

In [157]:
import spacy
nlp = spacy.load('en')
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

def predict_sentiment(model, sentence):
    model.eval()
    tokenized = [tok.text for tok in nlp.tokenizer(sentence)]
    #indexed = [custom_embeddings.stoi[t] for t in tokenized]
    indexed=[]
    for t in tokenized:
        try: indexed.append(custom_embeddings.stoi[t])
        except KeyError: indexed.append(custom_embeddings.stoi['<unk>'])
            
    length = [len(indexed)]
    tensor = torch.LongTensor(indexed).to(device)
    tensor = tensor.unsqueeze(1)
    length_tensor = torch.LongTensor(length)

    prediction = torch.sigmoid(model(tensor))
    return list(prediction.squeeze().detach())

In [158]:
predict_sentiment(model, "retaxrded quiet or kill you fucking stupid looser")

[tensor(0.9999),
 tensor(0.5946),
 tensor(0.9965),
 tensor(0.7356),
 tensor(0.9938),
 tensor(0.1544)]

In [156]:
dic= {1:'a' ,2 :'b'}
dic.items()

dict_items([(1, 'a'), (2, 'b')])